In [ ]:
DATA_DIR = '/home/pal.balazs/data/kelvin_helmholtz/'

In [ ]:
import os
import h5py

import numpy as np
from scipy.interpolate import griddata

import colorcet as cc
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
def get_keys(s):
    with h5py.File(s, "r") as f:
        keys = list(f['PartType0'].keys())
    return keys

In [ ]:
def get_data(s):
    with h5py.File(s, "r") as f:
        X, Y, _ = f['PartType0']['Coordinates'][:].T
        D = f['PartType0']['Density'][:]
        h = f['PartType0']['SmoothingLength'][:]
    return X, Y, D, h

In [ ]:
def get_grid_data(s, N):
    X, Y, D, _ = get_data(s)
    A = round(np.max(X) / np.max(Y), 1)
    xi, yi = np.meshgrid(np.linspace(0, np.max(X), int(N*A)),
                         np.linspace(0, np.max(Y), N),
                         indexing='ij')
    return griddata(np.stack((X, Y), axis=1), D, (xi, yi), method='nearest')

## Open and visualize snapshot

In [ ]:
run = os.path.join(DATA_DIR, 'kh2048')
fn = sorted([os.path.join(run, f) for f in os.listdir(run) if f.startswith('snapshot_')])

In [ ]:
len(fn)

In [ ]:
si = -1

In [ ]:
X, Y, D, _ = get_data(fn[si])
grid = get_grid_data(fn[si], N=2048)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis(False)

ax.imshow(np.rot90(np.nan_to_num(grid), axes=(-2,-1)),
          cmap=cm.RdBu_r)

#plt.savefig(f'{os.path.basename(run)}-field-snapshot{si}',
#            pad_inches=0.0, bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
top = mpl.colormaps['Oranges_r'].resampled(128)
bottom = mpl.colormaps['Blues'].resampled(128)

newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis(False)

#cmap = cm.get_cmap("cet_CET_CBD2")
ax.scatter(X, Y, c=D, cmap=newcmp, s=3)
ax.set_xlim(min(X), max(X))
ax.set_ylim(min(Y), max(Y))

plt.show()

In [ ]:
nr, nc = 2, 1
fig, axes = plt.subplots(nr, nc, figsize=(nc*6*2.7, nr*6*1), dpi=120,
                         facecolor='black')
fig.subplots_adjust(wspace=0, hspace=0)
axes = axes.flatten()

for ax in axes:
    ax.axis(False)
    ax.set_aspect('equal')

ax = axes[0]
ax.scatter(X, Y, c=D, cmap='viridis', s=0.5)
ax.set_xlim(min(X), max(X))
ax.set_ylim(min(Y), max(Y))

ax = axes[1]
ax.imshow(np.rot90(grid, axes=(-2,-1)))

plt.show()

## Datashader

In [ ]:
import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf

import matplotlib.cm as cm

In [ ]:
df = pd.DataFrame({'x': X, 'y': Y, 'd': D})

In [ ]:
%%time
n = 1.0
canvas = ds.Canvas(plot_width=int(n*851), plot_height=int(n*315))
tf.shade(canvas.points(df, 'x', 'y', ds.mean('d')),
         cmap=cm.inferno_r)

## Compare grid resolutions

In [ ]:
N = [128, 256, 512, 1024]
nr, nc = 2, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*6), dpi=120,
                         facecolor='black')
fig.subplots_adjust(wspace=0, hspace=0)
axes = axes.flatten()

for ax in axes:
    ax.axis(False)

for ax, ni in zip(axes, N):
    ax.imshow(np.rot90(get_grid_data(s, N=ni), axes=(-2,-1)),
              cmap='cividis_r')

plt.show()

## Animation

In [ ]:
def visualize_field(run, *,
                    fps=15, ni=512, cmap='viridis', fname=None):
    fig, ax = plt.subplots(figsize=(8, 8), dpi=120, facecolor='black')
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.set_aspect('equal')
    ax.axis(False)

    run = os.path.join(DATA_DIR, run)
    fn = sorted([os.path.join(run, f) for f in os.listdir(run) if f.startswith('snapshot_')])
    data = np.rot90(get_grid_data(fn[0], N=ni), axes=(-2,-1))
    img = ax.imshow(data, cmap=cmap, animated=True)

    def update(frame):
        data = np.rot90(get_grid_data(fn[frame], N=ni), axes=(-2,-1))
        img.set_array(data)
        return img,

    ani = FuncAnimation(fig, update, frames=len(fn), interval=1000/fps, blit=True)
    plt.show()

    # Save animation
    if fname is None:
        fname = f'{os.path.basename(run)}-anim-field-{fps}fps.mp4'
    ani.save(fname, fps=fps, writer='ffmpeg')
    # or save as gif
    # ani.save('wave_animation.gif', writer='imagemagick')

In [ ]:
%%time
run = 'kh2048'
fps = 24
ni = 2048
visualize_field(run, fps=fps, ni=ni, cmap=cm.RdBu_r)

In [ ]:
def visualize_sph(run, *,
                  fps=15, cmap='viridis', fname=None):
    fig, ax = plt.subplots(figsize=(8, 8), dpi=120, facecolor='black')
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.set_aspect('equal')
    ax.axis(False)

    run = os.path.join(DATA_DIR, run)
    fn = sorted([os.path.join(run, f) for f in os.listdir(run) if f.startswith('snapshot_')])
    X, Y, D, _ = get_data(fn[0])
    scatter = ax.scatter(X, Y, c=D, s=3.5, cmap=cmap, animated=True)
    ax.set_xlim(min(X), max(X))
    ax.set_ylim(min(Y), max(Y))

    def update(frame):
        X, Y, D, _ = get_data(fn[frame])
        scatter.set_offsets(np.c_[X, Y])
        scatter.set_array(D)
        return scatter,

    ani = FuncAnimation(fig, update, frames=len(fn), interval=1000/fps, blit=True)
    plt.show()

    # Save animation
    if fname is None:
        fname = f'{os.path.basename(run)}-anim-sph-{fps}fps.mp4'
    ani.save(fname, fps=fps, writer='ffmpeg')
    # or save as gif
    # ani.save('wave_animation.gif', writer='imagemagick')

In [ ]:
%%time
run = 'kh192'
visualize_sph(run, fps=24, cmap=newcmp)

RUPTURE.

The Kelvin-Helmholtz instability (KHI) is a physical phenomenon in (magneto)hydrodynamics that typically arises when two fluids of different velocities come in contact. The velocity difference (or more precisely velocity shear) causes the interface of the two fluids to become unstable, creating a vortex-like turbulence along their boundary.

The image above shows the density field of a snapshot from a 2D simulation of the KHI with periodic boundary conditions. The simulation consists of two inviscid (super)fluids with a density ratio of 1:2, flowing in the opposite directions with the same velocity. The denser fluid, which occupies the central portion of the image, is primarily illustrated in red, while the less dense areas are portrayed in blue. The turbulent boundary region where the fluids interact showcases a gradient transitioning from blue to red (through white in the middle). This color transition provides a visual representation of the merging fluids and the complex dynamics at play within the vortex features on the fluid boundary. Theoretically, this setup would result in an infinitely repeating fractal pattern that can also be seen here. A sheet of vortices form not just on the fluid boundary, but on the individual vortices too. A handful of these lesser vortices also show even smaller-scale vortices forming on them.

The hydrodynamic simulation was carried out using the GIZMO multi-physics code, utilizing its meshless finite-mass/finite-volume hydrosolver method developed by Hopkins (2015). Initial conditions with smooth boundaries were generated according to McNally et al. (2012) and Berlok (2019) on a grid of 5529x2048 particles (which is approximately the standard ratio of Facebook covers - 851:315 or approximately 2.7:1). The simulation was completed on the computational infrastructure of the Wigner RCP's GPU Laboratory using 128 CPU threads on an AMD EPYC 7742, 64-core processor in ~30 hours, which is around 160 days of total CPU time. The final visualization was made using matplotlib in Python.

---

Hopkins, P. F. (2015). A new class of accurate, mesh-free hydrodynamic simulation methods. Monthly Notices of the Royal Astronomical Society, 450(1), 53-110.

McNally, C. P., Lyra, W., & Passy, J. C. (2012). A well-posed Kelvin–Helmholtz instability test and comparison. The Astrophysical Journal Supplement Series, 201(2), 18.

Berlok, T., & Pfrommer, C. (2019). On the Kelvin–Helmholtz instability with smooth initial conditions–linear theory and simulations. Monthly Notices of the Royal Astronomical Society, 485(1), 908-923.

RUPTURE.

The Kelvin-Helmholtz instability (KHI) is a physical phenomenon in (magneto)hydrodynamics that typically arises when two fluids of different velocities come in contact. The velocity difference (or more precisely velocity shear) causes the interface of the two fluids to become unstable, creating a vortex-like turbulence along their boundary.

The image above shows the density field of a snapshot from a 2D simulation of the KHI with periodic boundary conditions. The simulation consists of two inviscid (super)fluids with a density ratio of 1:2, flowing in the opposite directions with the same velocity. The denser fluid, which occupies the central portion of the image, is primarily illustrated in red, while the less dense areas are portrayed in blue. The turbulent boundary region where the fluids interact showcases a gradient transitioning from blue to red (through white in the middle). This color transition provides a visual representation of the merging fluids and the complex dynamics at play within the vortex features on the fluid boundary. Theoretically, this setup would result in an infinitely repeating fractal pattern that can also be seen here. A sheet of vortices form not just on the fluid boundary, but on the individual vortices too. A handful of these lesser vortices also show even smaller-scale vortices forming on them.

The hydrodynamic simulation was carried out using the GIZMO multi-physics code, utilizing its meshless finite-mass/finite-volume hydrosolver method developed by Hopkins (2015). Initial conditions with smooth boundaries were generated according to McNally et al. (2012) and Berlok (2019) on a grid of 2048x2048 particles. The simulation was completed on the computational infrastructure of the Wigner RCP's GPU Laboratory using 128 CPU threads on an AMD EPYC 7742, 64-core processor in ~30 hours, which is around 160 days of total CPU time. The final visualization was made using matplotlib in Python.

---

Hopkins, P. F. (2015). A new class of accurate, mesh-free hydrodynamic simulation methods. Monthly Notices of the Royal Astronomical Society, 450(1), 53-110.

McNally, C. P., Lyra, W., & Passy, J. C. (2012). A well-posed Kelvin–Helmholtz instability test and comparison. The Astrophysical Journal Supplement Series, 201(2), 18.

Berlok, T., & Pfrommer, C. (2019). On the Kelvin–Helmholtz instability with smooth initial conditions–linear theory and simulations. Monthly Notices of the Royal Astronomical Society, 485(1), 908-923.

MOZAIC.

Yet another simulation of the Kelvin-Helmholtz instability with the same physical characteristics, but now with a much smaller resolution to visually give rise to the simualation's microscopic nature. Individual SPH particles on a meshless grid pushes against each other, creating a mozaic-like domain structure.

The hydrodynamic simulation was carried out using the GIZMO multi-physics code, utilizing its meshless finite-mass/finite-volume hydrosolver method developed by Hopkins (2015). Initial conditions with smooth boundaries were generated according to McNally et al. (2012) and Berlok (2019) on a grid of 192x192 particles. The simulation was completed on the computational infrastructure of the Wigner RCP's GPU Laboratory using 128 CPU threads on an AMD EPYC 7742, 64-core processor in ~3 minutes, which is ~6.4 hours of total CPU time. The final visualization was made using matplotlib in Python.

---

Hopkins, P. F. (2015). A new class of accurate, mesh-free hydrodynamic simulation methods. Monthly Notices of the Royal Astronomical Society, 450(1), 53-110.

McNally, C. P., Lyra, W., & Passy, J. C. (2012). A well-posed Kelvin–Helmholtz instability test and comparison. The Astrophysical Journal Supplement Series, 201(2), 18.

Berlok, T., & Pfrommer, C. (2019). On the Kelvin–Helmholtz instability with smooth initial conditions–linear theory and simulations. Monthly Notices of the Royal Astronomical Society, 485(1), 908-923.